# Project 4
The data for this project is real data provided to us by the HPI School of Design Thinking. You are given as input a table stored in CSV format in a file named ‘project4.csv’. This table has 321 rows and five columns. Each row corresponds
to a student and the columns are as follows:

< hash > < Sex > < Discipline > < Nationality > < Semester >

- The < hash > field contains a cryptographic hash of the student’s name (for privacy reasons).
- The sex field contains ‘m’ for male and ‘f’ for female.
- The Discipline field contains one of the following seven entries:
    ‘Business’, ‘Creative Disciplines’, ‘Engineering’, ‘Humanities’, ‘Life Sciences’, ‘Media’ or ‘Social Sciences’.
- The Nationality field contains one of 37 nationalities, depending on the selfreported nationality of the student.
- The Semester field contains the semester in which that student was enrolled. This is stored as a code that indicates the semester and year. For example, the students in Winter 2015 semester have WT-15 (for Winter Term), and the students enrolled in this semester have the code ST-17.

In [146]:
import imp
import numpy as np
import pandas as pd
import utils
from matplotlib import pyplot as plt
%matplotlib inline

students = pd.read_csv('project4.csv')

## Teaming 1 (Arbitrary teaming)

In [161]:
teaming1 = utils.process_semesters(students, utils.arbitrary_teaming)
utils.store_teaming(teaming1, "output/teaming1.out")

## Teaming 2 (Intra-team diversity)

In [164]:
imp.reload(utils)
teaming2 = utils.process_semesters(students, utils.intra_diversity_teaming)
utils.store_teaming(teaming2)  # "output/teaming2.out")

'"hash","team","Semester"\n'

## Teaming 3 (Inter-team diversity)

## Teaming 4 (Double inter-team diversity)